In [980]:
import numpy as np
import random as rd
import matplotlib.pyplot as plt
epochs = 100000

In [981]:
class Batch:
    quality = 0.5
    alpha = 1
    beta = 1

    def __init__(self, quality):
        self.quality = quality

    def sample_distribution(self):
        return np.random.beta(self.alpha, self.beta)

    def get_element(self):
        good = np.random.rand() < self.quality
        self.alpha = self.alpha + int(good)
        self.beta = self.beta + 1 - int(good)
        return good

In [982]:
class Factory:
    batches = []
    systems = 0
    good = 0

    def set_batches(self, qualities):
        self.batches = [Batch(quality) for quality in qualities]
        self.systems = 0
        self.good = 0

    def get_element(self):
        samples = [batch.sample_distribution() for batch in self.batches]
        choice = np.argmax(samples)
        result = self.batches[choice].get_element()
        self.systems = self.systems + 1
        self.good = self.good + int(result)

In [983]:
batches = [0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

factorium = Factory()
factorium.set_batches(batches)
for _ in range(epochs):
    factorium.get_element()
print(f'Score is {factorium.good / factorium.systems}')

Score is 0.94839


In [984]:
for idx in range(len(factorium.batches)):
    print(factorium.batches[idx].alpha, factorium.batches[idx].beta)

1 5
7 6
3 4
19 9
97 17
94718 5126


<h1><center><font size="10">First strategy</font></center></h1>

In [985]:
class Factory_if_bad_then_next:
    def __init__(self):
        self.batches = []
        self.systems = 0
        self.good = 0
        self.index = 0

    def set_batches(self, qualities):
        self.batches = [Batch(quality) for quality in qualities]
        self.systems = 0
        self.good = 0

    def get_element(self):
        result = self.batches[self.index].get_element()
        if not result:
            self.index += 1
            if self.index == len(self.batches):
                self.index = 0
        self.systems = self.systems + 1
        self.good = self.good + int(result)

In [986]:
factorium_ibtn = Factory_if_bad_then_next()
factorium_ibtn.set_batches(batches)
for _ in range(epochs):
    factorium_ibtn.get_element()
print(f'Score is {factorium_ibtn.good / factorium_ibtn.systems}')

Score is 0.85914


In [987]:
for idx in range(len(factorium.batches)):
    print(factorium.batches[idx].alpha, factorium.batches[idx].beta)

1 5
7 6
3 4
19 9
97 17
94718 5126


<h1><center><font size="10">Second strategy</font></center></h1>

In [988]:
class Factory_ranking_simple:
    def __init__(self, target_sample_count: int):
        self.batches = []
        self.systems = 0
        self.good = 0
        self.batch_ranking = dict()
        self.index = 0
        self.sample_count = 0
        self.target_sample_count = target_sample_count

    def set_batches(self, qualities):
        self.batches = [Batch(quality) for quality in qualities]
        self.systems = 0
        self.good = 0

    def rank_batches(self):
        for batch in range(len(self.batches)):
            self.batch_ranking[batch] = [0, 0, 0]

    def get_element(self):
        while self.sample_count < self.target_sample_count:
            if self.systems < (len(self.batches) * 2):
                result = self.batches[self.index].get_element()
                good_value = self.batch_ranking[self.index][0]
                sum_value = self.batch_ranking[self.index][1]

                self.batch_ranking[self.index] = [good_value + int(result),
                                                  sum_value + 1,
                                                  (good_value + int(result)) / (sum_value + 1)]

                self.systems = self.systems + 1
                self.good = self.good + int(result)

                self.index += 1
                if self.index == len(self.batches):
                    self.index = 0

                self.sample_count += 1

            else:
                sorted_best_ratios = sorted(self.batch_ranking.items(), key=lambda x:x[1][2])
                for n_elements_to_pick, batch_index in enumerate(sorted_best_ratios):
                    for _ in range(0, n_elements_to_pick + 1):
                        result = self.batches[batch_index[0]].get_element()
                        good_value = self.batch_ranking[batch_index[0]][0]
                        sum_value = self.batch_ranking[batch_index[0]][1]

                        self.batch_ranking[batch_index[0]] = [good_value + int(result),
                                                              sum_value + 1,
                                                              (good_value + int(result)) / (sum_value + 1)]
                        self.systems = self.systems + 1
                        self.good = self.good + int(result)
                        self.sample_count += 1

In [989]:
factorium_ranking_simple = Factory_ranking_simple(target_sample_count=epochs)
factorium_ranking_simple.set_batches(batches)
factorium_ranking_simple.rank_batches()
factorium_ranking_simple.get_element()
print(f'Score is {factorium_ranking_simple.good / factorium_ranking_simple.systems}')

Score is 0.8192952986581878


In [990]:
for idx in range(len(factorium.batches)):
    print(factorium_ranking_simple.batches[idx].alpha, factorium_ranking_simple.batches[idx].beta)

2338 2431
5725 3803
9999 4289
15232 3820
21485 2332
27168 1404


<h1><center><font size="10">Third strategy</font></center></h1>

In [991]:
class Factory_ranking_hard:
    def __init__(self, target_sample_count: int):
        self.batches = []
        self.systems = 0
        self.good = 0
        self.batch_ranking = dict()
        self.index = 0
        self.sample_count = 0
        self.target_sample_count = target_sample_count
        self.ratios = []

    def set_batches(self, qualities):
        self.batches = [Batch(quality) for quality in qualities]
        self.systems = 0
        self.good = 0

    def rank_batches(self):
        for batch in range(len(self.batches)):
            self.batch_ranking[batch] = [0, 0, 0]

    def get_element(self):
        while self.sample_count < self.target_sample_count:
            if self.systems < (len(self.batches) * 2):
                result = self.batches[self.index].get_element()
                good_value = self.batch_ranking[self.index][0]
                sum_value = self.batch_ranking[self.index][1]

                self.batch_ranking[self.index] = [good_value + int(result),
                                                  sum_value + 1,
                                                  (good_value + int(result)) / (sum_value + 1)]

                self.systems = self.systems + 1
                self.good = self.good + int(result)

                self.index += 1
                if self.index == len(self.batches):
                    self.index = 0

                self.sample_count += 1

            else:
                sorted_best_ratios = sorted(self.batch_ranking.items(), key=lambda x:x[1][2])
                self.ratios.append(1)
                for batch in range(len(sorted_best_ratios) - 1):
                    try:
                        self.ratios.append(int(sorted_best_ratios[batch + 1][1][2] / sorted_best_ratios[batch][1][2]))
                    except ZeroDivisionError:
                        self.ratios.append(1)

                value = 1
                for position, batch_index in enumerate(sorted_best_ratios):
                    value *= self.ratios[position]
                    for _ in range(0, value):
                        result = self.batches[batch_index[0]].get_element()
                        good_value = self.batch_ranking[batch_index[0]][0]
                        sum_value = self.batch_ranking[batch_index[0]][1]

                        self.batch_ranking[batch_index[0]] = [good_value + int(result),
                                                              sum_value + 1,
                                                              (good_value + int(result)) / (sum_value + 1)]
                        self.systems = self.systems + 1
                        self.good = self.good + int(result)
                        self.sample_count += 1

In [992]:
factorium_ranking_hard = Factory_ranking_hard(target_sample_count=epochs)
factorium_ranking_hard.set_batches(batches)
factorium_ranking_hard.rank_batches()
factorium_ranking_hard.get_element()
print(f'Score is {factorium_ranking_hard.good / factorium_ranking_hard.systems}')

Score is 0.7806043879122417


In [993]:
for idx in range(len(factorium_ranking_hard.batches)):
    print(factorium_ranking_hard.batches[idx].alpha, factorium_ranking_hard.batches[idx].beta)

4981 5022
6012 4005
13972 6016
16018 3984
18091 1911
18994 1008


<h1><center><font size="10">Own strategy</font></center></h1>

In [994]:
class Factory_roulette:
    def __init__(self, target_sample_count: int):
        self.batches = []
        self.systems = 0
        self.good = 0
        self.batch_ranking = dict()
        self.index = 0
        self.sample_count = 0
        self.target_sample_count = target_sample_count
        self.ratios = []

    def set_batches(self, qualities):
        self.batches = [Batch(quality) for quality in qualities]
        self.systems = 0
        self.good = 0

    def rank_batches(self):
        for batch in range(len(self.batches)):
            self.batch_ranking[batch] = [0, 0, 0]

    def get_element(self):
        while self.sample_count < self.target_sample_count:
            if self.systems < int(self.target_sample_count * 0.01):
                result = self.batches[self.index].get_element()
                good_value = self.batch_ranking[self.index][0]
                sum_value = self.batch_ranking[self.index][1]

                self.batch_ranking[self.index] = [good_value + int(result),
                                                  sum_value + 1,
                                                  (good_value + int(result)) / (sum_value + 1)]

                self.systems = self.systems + 1
                self.good = self.good + int(result)

                self.index += 1
                if self.index == len(self.batches):
                    self.index = 0

                self.sample_count += 1

            else:
                sorted_best_ratios = sorted(self.batch_ranking.items(), key=lambda x:x[1][2])
                keys, values = list(), list()
                for index in range(len(sorted_best_ratios)):
                    keys.append(sorted_best_ratios[index][0])
                    values.append(sorted_best_ratios[index][1])

                choice = rd.choices(keys, weights=[value[2] * (index + 1) for index, value in enumerate(values)])[0]
                result = self.batches[choice].get_element()
                good_value = self.batch_ranking[choice][0]
                sum_value = self.batch_ranking[choice][1]

                self.batch_ranking[choice] = [good_value + int(result),
                                              sum_value + 1,
                                              (good_value + int(result)) / (sum_value + 1)]

                self.systems = self.systems + 1
                self.good = self.good + int(result)
                self.sample_count += 1

In [995]:
factorium_roulette = Factory_roulette(target_sample_count=epochs)
factorium_roulette.set_batches(batches)
factorium_roulette.rank_batches()
factorium_roulette.get_element()
print(f'Score is {factorium_roulette.good / factorium_roulette.systems}')

Score is 0.84106


In [996]:
for idx in range(len(factorium_roulette.batches)):
    print(factorium_roulette.batches[idx].alpha, factorium_roulette.batches[idx].beta)

1569 1570
4232 2738
8512 3635
14852 3749
23442 2572
31505 1636
